Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [12]:
''' First - adding it the stochastic logistic regression done on 2-fullyconnected '''
batch_size = 128

graph = tf.Graph()
with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    beta_val = tf.placeholder(tf.float32)  
    weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    loss = loss + beta_val * tf.nn.l2_loss(weights)
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [16]:
num_steps = 3001
beta = 0.01 # constant - added 0.3% accuracy
beta = 0.1 # constant - added 0.7% accuracy

def runSession(beta):
    with tf.Session(graph=graph) as session:
      tf.global_variables_initializer().run()
      print("Initialized")
      for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, beta_val : beta}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
            print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))
beta_vals = np.logspace(-4, -2, 20)
for b in beta_vals:
    print('-----------')
    print('beta:',b)
    runSession(b)

-----------
beta: 0.0001
Initialized
Minibatch loss at step 0: 15.981618
Minibatch accuracy: 10.2%
Validation accuracy: 9.8%
Test accuracy: 9.4%
Minibatch loss at step 500: 1.503785
Minibatch accuracy: 77.3%
Validation accuracy: 75.8%
Test accuracy: 83.6%
Minibatch loss at step 1000: 1.714602
Minibatch accuracy: 75.8%
Validation accuracy: 76.1%
Test accuracy: 84.1%
Minibatch loss at step 1500: 0.843820
Minibatch accuracy: 85.2%
Validation accuracy: 77.0%
Test accuracy: 85.1%
Minibatch loss at step 2000: 1.036397
Minibatch accuracy: 82.8%
Validation accuracy: 77.6%
Test accuracy: 85.4%
Minibatch loss at step 2500: 1.181303
Minibatch accuracy: 77.3%
Validation accuracy: 78.6%
Test accuracy: 86.6%
Minibatch loss at step 3000: 1.089244
Minibatch accuracy: 77.3%
Validation accuracy: 79.2%
Test accuracy: 86.6%
-----------
beta: 0.00012742749857031334
Initialized
Minibatch loss at step 0: 17.606117
Minibatch accuracy: 8.6%
Validation accuracy: 9.3%
Test accuracy: 9.2%
Minibatch loss at step 5

Initialized
Minibatch loss at step 0: 20.230251
Minibatch accuracy: 14.1%
Validation accuracy: 15.8%
Test accuracy: 16.3%
Minibatch loss at step 500: 2.567128
Minibatch accuracy: 76.6%
Validation accuracy: 76.1%
Test accuracy: 83.9%
Minibatch loss at step 1000: 1.859608
Minibatch accuracy: 79.7%
Validation accuracy: 79.0%
Test accuracy: 86.5%
Minibatch loss at step 1500: 0.848027
Minibatch accuracy: 86.7%
Validation accuracy: 80.0%
Test accuracy: 87.5%
Minibatch loss at step 2000: 0.804425
Minibatch accuracy: 86.7%
Validation accuracy: 80.8%
Test accuracy: 87.8%
Minibatch loss at step 2500: 0.836356
Minibatch accuracy: 78.1%
Validation accuracy: 81.6%
Test accuracy: 88.8%
Minibatch loss at step 3000: 0.733899
Minibatch accuracy: 82.8%
Validation accuracy: 81.8%
Test accuracy: 88.8%
-----------
beta: 0.0014384498882876629
Initialized
Minibatch loss at step 0: 23.201576
Minibatch accuracy: 8.6%
Validation accuracy: 14.6%
Test accuracy: 15.3%
Minibatch loss at step 500: 2.624696
Minibatch

In [20]:
''' Adding the l2-loss func to the nn with 1 hidden layer from 2-fullyconnected '''
num_steps = 3001
batch_size = 128
hidden_nodes = 1024
image_size = 28
num_labels = 10

graph = tf.Graph()
with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    beta_val = tf.placeholder(tf.float32)
    # Variables.
    #   first we have matrix that is from the size of the input (image_size^2) to the size of the hidden nodes
    w_1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes]))
    b_1 = tf.Variable(tf.zeros([hidden_nodes]))
    #   second we have matrix that is from the size of the hidden nodes to the size of the inputs
    w_2 = tf.Variable(tf.truncated_normal([hidden_nodes, num_labels]))
    b_2 = tf.Variable(tf.zeros([num_labels]))

    def ForwardProp(dataset):
        z1 = tf.matmul(dataset, w_1) + b_1 # first variables
        a1 = tf.nn.relu(z1) # activation function ReLU
        z2 = tf.matmul(a1, w_2) + b_2
        return z2

    # Training computation.
    logits = ForwardProp(tf_train_dataset)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) \
        + beta_val * ( tf.nn.l2_loss(w_1) + tf.nn.l2_loss(w_1) + tf.nn.l2_loss(b_1) + tf.nn.l2_loss(b_2))

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax( ForwardProp(tf_valid_dataset) )
    test_prediction = tf.nn.softmax( ForwardProp(tf_test_dataset) )

In [23]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, beta_val : 0.0001}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 501.900604
Minibatch accuracy: 5.5%
Validation accuracy: 32.7%
Minibatch loss at step 500: 72.086235
Minibatch accuracy: 82.0%
Validation accuracy: 81.4%
Minibatch loss at step 1000: 59.223553
Minibatch accuracy: 80.5%
Validation accuracy: 80.4%
Minibatch loss at step 1500: 47.796803
Minibatch accuracy: 89.1%
Validation accuracy: 81.5%
Minibatch loss at step 2000: 41.902218
Minibatch accuracy: 91.4%
Validation accuracy: 82.1%
Minibatch loss at step 2500: 37.965393
Minibatch accuracy: 84.4%
Validation accuracy: 82.6%
Minibatch loss at step 3000: 34.625271
Minibatch accuracy: 82.0%
Validation accuracy: 83.6%
Test accuracy: 90.5%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---
